This file preserve python code I used for task 2

### Q1: How is the bike share business in San Francisco performing over time?

In [1]:
# comparing subscriber and customer loyalty

### Q2: What are your observations when you compare bike share users in San Francisco with bike share users in New York?

In [ ]:
# comparing temperature effect 